# nnUNet Segmentation Service Client

This Jupyter Notebook provides an example on how to interact with a running nnUNet Segmentation service

### Getting Started:
1. Set the host name or IP address of the server running the service
2. Set the appropriate port which the service is running on
3. Create an API key on the server for this client and enter it below
4. Leave the algorithm name, since this should always be running as 'nnUNet Segmentation'
5. While experimenting, set the log_level to 'DEBUG'. This will generate a lot of output so set it to 'INFO' when running over a lot of data to reduce the amount of output

In [ ]:
import sys
sys.path.append("../../..")

import os
from pathlib import Path

import SimpleITK as sitk    
import time

from platipy.backend.client import PlatiPyClient
from platipy.imaging.tests.data import get_lung_nifti
from platipy.imaging import ImageVisualiser
from platipy.imaging.label.utils import get_com

from loguru import logger

host = "127.0.0.1" # Set the host name or IP of the server running the service here
port = 8001 # Set the port the service was configured to run on here

api_key = 'XXX' # Put API key here

algorithm_name = "nnUNet Service" # The name of the algorithm, in this case it should be left as is

log_level = "INFO" # Choose an appropriate level of logging output: "DEBUG" or "INFO"

logger.remove()
handler_id = logger.add(sys.stderr, level=log_level)

### Fetch some data

The next cell fetches some test data to work with from TCIA. We can use this as our atlas for this example as well as use one of the images to infer the auto-segmentation.

In [ ]:
lung_data = get_lung_nifti()

### Create Client Instance

The PlatiPyClient provides useful functions to interact with the running service

In [ ]:
client = PlatiPyClient(host, port, api_key, algorithm_name)

### Add a dataset

Images can only be added to a dataset. In theory you could add multiple images to one dataset and the algorithm would run on all of them. But often better control can be gained by only adding one image per dataset and runnin the algorithm on each separately.

In [ ]:
dataset = client.add_dataset()

### Add an image to the dataset

The following cell grabs the first test image file and adds it as a data object to the dataset created above.

This is the image that will be inferred by the service. We will configure the path to the atlas below.

In [ ]:
images = [i for i in lung_data.glob("*/IMAGES/*.nii.gz")]
ct_image = str(images[0])
data_object = client.add_data_object(dataset, file_path=ct_image)

### Refresh the dataset

The next cell demonstrates how to refresh the dataset and see that the image has been added as a input data object

In [ ]:
client.get_dataset(dataset)

### View and modify the algorithm configuration

Here we can modify the default settings for the algorithm. There are a number of settings which can be modified. Here the path to the atlas 

In [ ]:
settings = client.get_default_settings()
settings['task'] = "Task200_ClinicalHeart"
settings['config'] = "3d_lowres"
settings['trainer'] = "nnUNetTrainerHeart"
settings['clean_sup_slices'] = True

### Run the algorithm

Now everything is ready to run the algorithm using the dataset and the modified settings we generated above

In [ ]:
start = time.time()

for status in client.run_algorithm(dataset, config=settings):
    print('.', end='')

end = time.time()
print(f"Took {end - start:.1f} seconds")

### Download the output

Once the algorithm has finished running, we can download the output objects (here downloaded into the results directory)r.json()

In [ ]:
output_directory = os.path.join(".", "results")
client.download_output_objects(dataset, output_path=output_directory)

### Display the results

In [ ]:
heart = sitk.ReadImage(str([s for s in Path(output_directory).glob("*")][0]))

vis = ImageVisualiser(sitk.ReadImage(str(ct_image)), cut=get_com(heart))
vis.add_contour({"Heart": heart})
fig=vis.show()